In [1]:
import sys
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='image.jpg' 
# os.rename(list(uploaded.keys())[0], dst)

In [3]:
# import os
# from google.colab import files
# uploaded = files.upload()
# dst ='movie.mp4' 
# os.rename(list(uploaded.keys())[0], dst)

In [4]:
# damage colors(BGR)
elementColors = dict()
elementColors['anemo'] = (200, 255, 106)
elementColors['geo'] = (101, 204, 254)
elementColors['electro'] = (255, 155, 224)
elementColors['physical'] = (255, 255, 255)
elementColors['pyro'] = (3, 155, 254)
elementColors['cyro'] = (255, 252, 155)
elementColors['hydro'] = (253, 204, 52)

resizeScale = 0.5
colorKey = "cyro"

In [5]:
def rectangleScaled(im, start, end, color, thickness=1):
    start = (int(start[0] / resizeScale), int(start[1] / resizeScale))
    end = (int(end[0] / resizeScale), int(end[1] / resizeScale))
    cv2.rectangle(im, start, end, color, thickness)

def restoreRectangle(rect):
    return (int(rect[0] / resizeScale), int(rect[1] / resizeScale), int(rect[2] / resizeScale), int(rect[3] / resizeScale))

In [6]:
templates = []
for i in range(10):
  templates.append(cv2.resize(cv2.imread('./templates/' + str(i) + '.png', 0), (36, 36)))

def matchingNumber(im):
  maxVal_All = -1
  num_dsp = 0
  resizedIm = cv2.resize(im, (36, 36))
  for i in range(10):

    result= cv2.matchTemplate(resizedIm,templates[i],cv2.TM_CCOEFF_NORMED)
    mmr=cv2.minMaxLoc(result)
    maxVal=mmr[1]    
        
    if maxVal > maxVal_All:
        num_dsp = i
        maxVal_All = maxVal
    
  return num_dsp
            


In [7]:
def getNumber(original, resized, rect):
  global imageCounter
  x, y, w, h = restoreRectangle(rect)
  height = 150
  scale = height / h
  partImage = cv2.resize(original[y:y+h, x:x+w, :], (int(height / h * w), height))
  
  rangeW = 25
  color = elementColors[colorKey]
  partImage = cv2.inRange(partImage, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))

  closing = cv2.dilate(partImage,(2, 2),iterations = 1)
  thresh = cv2.adaptiveThreshold(closing, 255, 1, 1, 5, 2)
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  if len(contours) < 4:
    return 0
  lists = []
  for cnt in contours:
    nextRect = cv2.boundingRect(cnt)
    x2, y2, w2, h2 = nextRect
    if h2 < 0.8 * height or w2 / h2 > 1:
      continue
    red = (255, 0, 0)
    # cv2.imwrite("./output/{}.png".format(imageCounter), closing[y2:y2+h2,x2:x2+w2])
    num = matchingNumber(closing[y2:y2+h2,x2:x2+w2])
    lists.append((num, x2))
    imageCounter += 1
    cv2.rectangle(original, (x + int(x2 / scale), y + int(y2 / scale)), (x + int((x2 + w2) / scale), y + int((y2 + h2) / scale)), red, 2)
  
  lists.sort(key=lambda x: -x[1])
  total = 0
  for i, value in enumerate(lists):
    total += value[0] * (10 ** i) 

  return total

In [8]:
imageCounter = 0
def outputNumberBinaryImage(im, rect):
  global imageCounter
  x,y,w,h = rect
  outputImage = im[y:y+h, x:x+w]
  closing = cv2.dilate(outputImage,(4, 4),iterations = 1)
  cv2.imwrite("./output/{}_{}.png".format(colorKey, imageCounter), closing)
  imageCounter += 1

In [9]:


def drawContours(im, resized):
  global prevFrameNumber, resizeScale, textpos
  
  rangeW = 25
  color = elementColors[colorKey]
  inrange = cv2.inRange(resized, (max(color[0] - rangeW, 0), max(color[1] - rangeW, 0), max(color[2] - rangeW, 0)), (min(255, color[0] + rangeW), min(255, color[1] + rangeW), min(255, color[2] + rangeW)))
  # for key, color in elementColors.items():
  #   nextInrange = cv2.inRange(im, (color[0] - rangeW, color[1] - rangeW, color[2] - rangeW), (color[0] + rangeW, color[1] + rangeW, color[2] + rangeW))
  #   inrange = np.maximum(nextInrange, inrange)

  # blur = cv2.GaussianBlur(inrange, (5, 5), 0)
  thresh = cv2.adaptiveThreshold(inrange, 255, 1, 1, 7, 2)

  contours, _ = cv2.findContours(
      thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  frameContours = []
  
  nextFrameNumber = []

  for cnt in contours:
    rect = cv2.boundingRect(cnt.astype(np.int32))
    x, y, w, h = rect
    frameContours.append((x + w / 2, y + h / 2))
    

    if h < 2 or w / (h * 0.8) < 3:
      continue
    # flag = False
    # for px, py in prevFrameContours:
    #   if abs(px - (x + w / 2)) < 50 and abs(py - (y - h / 2)) < 50:
    #     flag = True
    #     break
    # if flag:
    #   continue
    
    # outputNumberBinaryImage(inrange, rect)
    red = (0, 0, 255)
    num = getNumber(im, inrange, rect)
    if num < 1000:
      continue
    rectangleScaled(im, (x, y), (x + w, y + h), red, 2)
    cv2.putText(im, str(num), (int((x + w/2) / resizeScale), int((y+h * 2) / resizeScale)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 5)
    nextFrameNumber.append(num)

    prevFrameExist = False
    for prevNum in prevFrameNumber:
      if num == prevNum:
        prevFrameExist = True
        break
    
    if not prevFrameExist:
        textQueue.put(str(num))
        if (textQueue.qsize() == 20):
          textQueue.get()
        break
  
  prevFrameNumber = nextFrameNumber
  index = 0
  for index, text in enumerate(list(textQueue.queue)):
    cv2.putText(im, text, (0, index * 50 + 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 5)
    index += 1
  
  prevFrameContours = frameContours

  return im

In [11]:
filename = "fast-yoimiya"
cap = cv2.VideoCapture('./{}.mp4'.format(filename))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./{}_output.mp4'.format(filename), fourcc, frame_rate, (width, height))
colorKey = "pyro"
prevFrameNumber = []
import queue
textQueue = queue.Queue()

count = 0
imageCounter = 0

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

while(cap.isOpened()):
    ret, frame = cap.read()
    count += 1
    if count % 24 == 0:
      print(count / frame_count * 100, '%')
    if (not(ret)):
      break
    
    resized = cv2.resize(frame, (int(frame.shape[1] * resizeScale), int(frame.shape[0] * resizeScale)))
    res = drawContours(frame, resized)

    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    out.write(res)
    cv2.imshow("frame", res)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()
    


0.15965939329430548 %
0.31931878658861096 %
0.4789781798829164 %
0.6386375731772219 %
0.7982969664715274 %
0.9579563597658328 %
1.1176157530601383 %
1.2772751463544438 %
1.4369345396487492 %
1.5965939329430547 %
1.7562533262373603 %
1.9159127195316656 %
2.0755721128259714 %
2.2352315061202765 %
2.394890899414582 %
2.5545502927088877 %
2.7142096860031932 %
2.8738690792974984 %
3.0335284725918044 %
3.1931878658861095 %
3.352847259180415 %
3.5125066524747206 %
3.6721660457690266 %
3.8318254390633313 %
3.991484832357637 %
4.151144225651943 %
4.310803618946248 %
4.470463012240553 %
4.630122405534859 %
4.789781798829164 %
4.94944119212347 %
5.109100585417775 %
5.2687599787120805 %
5.4284193720063865 %
5.5880787653006925 %
5.747738158594997 %
5.907397551889303 %
6.067056945183609 %
6.226716338477914 %
6.386375731772219 %
6.546035125066525 %
6.70569451836083 %
6.865353911655135 %
7.025013304949441 %
7.184672698243746 %
7.344332091538053 %
7.5039914848323575 %
7.663650878126663 %
7.823310271420